# 数值计算方法

2024 年秋季学期本科

## 第 3 章 方程求根

### 引言

#### 案例导入：人口增长问题

人口增长通常被假设为连续地依赖于时间，并且与当前人口数成正比. 为此，建立数学模型. 

$$\frac{{\rm{d}}N(t)}{{\rm{d}}t}=r N(t)$$

式中，$N(t)$ 是种群大小，即某一特定时间内种群的个体数量. 常数 $r$ 是人均增长率（人口中每个成员对人口增长的平均贡献）. 

容易求得这个微分方程的解是 $N(t)=N_0e^{rt}$，其中$N_0$ 是初始种群数量. 由此得到人口增长的指数模型.

但是，人群并不是生活在孤立的系统，因此要考虑种群迁徙. 

修改方程，使其纳入一个表示迁徙的项 $v$，比如

$$\frac{{\rm{d}}N(t)}{{\rm{d}}t}=r N(t)+v$$

式中，$v$ 是假设的常数迁徙率. 则方程的解变成

$$N(t)=N_0e^{r t}+\frac{v}{r}(e^{r t}-1)$$

可以通过数据来**率定** (calibration) 模型中出现的未知参数. 我们代入具体数据，作如下假设

- $N(0)=1000000$（人）
- $N(1)=1564000$（人）
- $v=435000$（人/年）
- $\Delta t = 1$（年）

得到关于未知数 $r$ 的一个方程

$$1564000 = 1000000 e^r + \frac{435000}{r}(e^r-1)$$

显然，这是一个超越方程，无法直接求得方程的根.

求解关于 $x$ 的方程 $f(x) = 0$ 的**根** (root) 或解. 当明确的分析解不可能实现时，所有的方法都是通过计算一个序列 $x_0, x_1, x_2, \dots$，收敛于满足 $f(r)=0$ 的根 $x = r$. 

方程的根，也叫函数 $f$ 的**零点** (zero).

**求根问题** (root-finding problem) 是数值逼近中最基本的问题之一.

In [1]:
import math

test_func = lambda r : math.exp(r) + 0.435 * (math.exp(r)-1) / r - 1.564

### 二分法

#### 理论

**二分法** (bisection method) 基于**介值定理** (Intermediate Value Theorem) 和计算机算法的**二分搜索** (binary search). 

假设 $f$ 是一个定义在区间 $[a, b]$ 上的连续函数，$f(a)$ 和 $f(b)$ 异号. 介值定理确保在 $(a, b)$ 中存在一个数 $p$，使得 $f(p)=0$. 

> 介值定理：假设$f\in C[a,b]$且$K$是介于$f(a)$和$f(b)$之间的一个数，则存在$c\in (a,b)$，满足$f(c)=K$.

为了简单起见，假设该区间内的根唯一. 

二分法对 $[a, b]$ 的子区间反复减半，并在每一步中找到包含有解的那一半区间. 

记 $a_1=a, b_1=b$, 则区间 $[a_1, b_1]$ 的中点是 $p_1=\displaystyle\frac{a_1+b_1}{2}$. 中点划分之后新形成两个区间 $[a_1, p_1]$ 和 $[p_1,b_1]$，把其中含有解 $p$ 的那一半记为 $[a_2, b_2]$，如此迭代下去，形成一系列**区间套** (nested interval) $$[a_1, b_1]\supset[a_2, b_2]\supset\cdots\supset[a_k, b_k]\supset\cdots$$

<img src="./assets/bisection.png" width="480" style="display: block;margin-left: auto;margin-right: auto;" />

满足以下任意一种停机条件均可以使迭代终止

- $|p_N-p_{N-1}|<\varepsilon$
- $\displaystyle\frac{ |p_N-p_{N-1} }{ |p_N|}|<\varepsilon\quad(p_N\ne0)$
- $|f(p_N)|<\varepsilon$

此外，为算法设置一个最大迭代次数，这样可以确保程序不会陷入无限循环.

- 二分法在概念上是最简单的方法，几乎总是有效. 
- 然而，它也是最慢的方法，大多数时候应该避免使用.

In [2]:
def bisection_method(f, a, b, tau, N):
    if f(a) * f(b) >= 0:
        print("失败：f(a)和f(b)必须有不同的符号")
        return None

    for n in range(1, N+1):
        p = (a + b) / 2
        fp = f(p)
        
        if abs(fp) < tau:
            return p
        elif f(a) * fp < 0:
            b = p
        else:
            a = p
    
    print("失败：达到最大迭代次数")
    return None

# 指定区间进行测试
eps = 1e-5
root = bisection_method(test_func, eps, 2, eps, 1000)
if root is not None:
    print(f"找到根：x = {root}")

找到根：x = 0.10099216110229492


In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

class RootFindingAnimator:
    def __init__(self, f, method, x0, tau, N):
        """
        初始化动画类
        :param f: 目标函数
        :param method: 使用的求根算法函数，需要返回根及每次迭代的点
        :param x0: 初始猜测值
        :param tau: 精度要求
        :param N: 最大迭代次数
        """
        self.f = f
        self.method = method
        self.x0 = x0
        self.tau = tau
        self.N = N
        self.root, self.iterations = self.method(self.f, self.x0, self.tau, self.N)
    
    def animate(self):
        """
        生成并显示动画
        """
        fig, ax = plt.subplots()
        x = np.linspace(min(self.iterations)-1, max(self.iterations)+1, 400)
        y = self.f(x)
        ax.plot(x, y, 'b-', label=f"f(x)")
        ax.axhline(0, color='black', linewidth=0.5)

        line, = ax.plot([], [], 'ro', label="Current Estimate")
        text = ax.text(0.05, 0.95, '', transform=ax.transAxes, ha='left', va='top')

        def init():
            line.set_data([], [])
            text.set_text('')
            return line, text

        def update(frame):
            x_n = self.iterations[frame]
            line.set_data(x_n, self.f(x_n))
            text.set_text(f"Iteration {frame+1}\nx = {x_n:.5f}")
            return line, text

        ani = FuncAnimation(fig, update, frames=len(self.iterations), init_func=init, blit=True, repeat=False)

        plt.close(fig)  # 关闭静态图像

        return HTML(ani.to_jshtml())

def bisection_method(f, a, b, tau, N):
    if f(a) * f(b) >= 0:
        print("失败：f(a)和f(b)必须有不同的符号")
        return None, []

    iterations = []
    for n in range(1, N+1):
        p = (a + b) / 2
        iterations.append(p)  # 记录每次迭代的中点
        
        fp = f(p)
        if abs(fp) < tau:
            return p, iterations
        elif f(a) * fp < 0:
            b = p
        else:
            a = p
    
    print("失败：达到最大迭代次数")
    return None, iterations

# 示例目标函数
def test_func(x):
    return x**2 - 2

a, b = 0, 2
eps = 1e-5
N = 1000

# 使用RootFindingAnimator类
animator = RootFindingAnimator(test_func, lambda f, x0, tau, N: bisection_method(f, a, b, tau, N), a, eps, N)
animator.animate()

### 不动点迭代法

如果 $g(p)=p$，则数 $p$ 是一个给定函数 $g$ 的**不动点** (fixed point).

如果函数 $g$ 在 $p$ 处有一个不动点，那么函数
$$ f(x) = x - g(x)$$
在 $p$ 处有一个零点.

> 【例 3.1】
>
> 找到函数 $g(x)=x^2-2$的不动点.

根据不动点的性质，有

$$p=g(p)=p^2-2$$

移项做因式分解，有

$$0=p^2-p-2=(p+1)(p-2)$$

因此有两个不动点，分别是 $p=-1$ 和 $p=2$. 函数 $g$ 的不动点出现在 $y=g(x)$ 和 $y=x$ 图像的交点处.

> **不动点存在且唯一的充分条件**：如果 $g \in C[a,b]$ 且对于所有 $x \in [a,b]$ 都有 $g(x) \in [a,b]$，那么 $g$ 在 $[a,b]$ 中至少有一个不动点. 此外，如果 $g^\prime(x)$ 存在于 $(a,b)$ 上，并且存在一个常数 $0<L<1$，使得对于所有 $x \in (a,b)$ 有 $|g^\prime(x)|\le L$，那么在 $[a,b]$ 中有且仅有一个不动点.
>
> 【证明】如果 $g \in C[a,b]$ 且对于所有 $x \in [a,b]$ 都有 $g(x) \in [a,b]$，那么 $g$ 在 $[a,b]$ 中至少有一个不动点. 
>
> 如果 $g(a)=a$ 或 $g(b)=b$ ，那么 $g$ 在区间端点上有一个不动点. 
>
> 如果不是，那么 $g(a)>a$，$g(b)<b$. 函数 $h(x)=g(x)-x$ 在 $[a, b]$ 上是连续的，$h(a)=g(a)-a>0$ 且 $h(b)=g(b)-b<0$. 由介值定理可知，存在 $p \in (a, b)$，对其而言，$h(p) = 0$. 数 $p$ 是 $g$ 的不动点，因为 $0=h(p)=g(p)-p$. 故而 $g(p)=p$.
> 假设 $|g^\prime(x)|\le L < 1$，$p$ 和 $q$ 都是 $[a, b]$ 中的不动点. 如果 $p \ne q$，那么由中值定理可知在 $p$ 和 $q$ 之间存在一个数 $\xi \in[a, b]$，满足 $$\frac{g(p)-g(q)}{p-q} =g^\prime(\xi)$$ 因此 $$|p-q|=|g(p)-g(q)|=|g^\prime(\xi)||p-q|\le L|p-q|<|p-q|$$这导致矛盾，矛盾来自唯一的假设，即 $p\ne q$. 因此 $p=q$，$[a,b]$中的不动点是唯一的.

为了逼近函数 $g$ 的不动点，选择一个初始近似值 $p_0$，并对每个 $k\ge 1$通过让 $p_k=g(p_{k-1})$ 产生序列 $\{p_k\}_{k=0}^\infty$. 如果序列收敛到 $p$，并且 $g$ 是连续的，那么 $$p= \lim_{k\to\infty} p_k = \lim_{k\to\infty} g(p_{k-1})=g\left(\lim_{k\to\infty} p_{k-1} \right)=g(p)$$就得到了 $x = g(x)$ 的解. 这种技术被称为**不动点迭代** (fixed-point iteration).

<img src="./assets/fixed-point.png" width="900" style="display: block;margin-left: auto;margin-right: auto;" />

> 【例 3.2】
>
> 非线性方程 $f(x)=x^3-x-1=0$ 的4种不同的迭代格式：
>
> - $g_1(x)=x^3-1$，因此 $x_{k+1}=x_k^3-1$
> - $g_2(x)=\sqrt[3]{x+1}$，因此 $x_{k+1}=\sqrt[3]{x_k+1}$
> - $g_3(x)=\displaystyle\frac{1}{x^2-1}$，因此 $x_{k+1}=\displaystyle\frac{1}{x_k^2-1}$
> - $g_4(x)=x-\displaystyle\frac{x^3-x-1}{3x^2-1}$，因此 $x_{k+1}=x_k-\displaystyle\frac{x_k^3-x_k-1}{3x_k^2-1}$
>
> 分别计算在不同迭代格式下，$f(x)=0$ 在 $[1,2]$ 内的解

解析解是 $$x=\frac{\sqrt[3]{9-\sqrt{69}}+\sqrt[3]{9+\sqrt{69}}}{\sqrt[3]{2}\cdot3^{2/3}}$$

|| $$x_{k+1}=g_1(x_k)$$ | $$x_{k+1}=g_2(x_k)$$ | $$x_{k+1}=g_3(x_k)$$ | $$x_{k+1}=g_4(x_k)$$|
|--|--|--|--|--|
|$x_0$ | 1.5                | 1.5                |-1.5      | 1.5                |
|$x_1$ | 2.375              | 1.357209           | -0.800000| 1.34782608695652   |
|$x_2$ | 12.396             | 1.330861           | -2.777778| 1.32520039895091   |
|$x_3$ | 1904.003           | 1.325884           | -0.148897| 1.32471817399905   |
|$x_4$ |                    | 1.324939           | -1.022673| 1.32471795724479   |
|$x_5$ |                    | 1.324760           | 21.805462| 1.32471795724475   |
|$x_6$ |                    | 1.324726           | -0.002108| 1.32471795724475   |

In [4]:
def fixed_point_iteration(g, x_0, tau, N):
    x_n = x_0
    for n in range(N):
        try:
            x_next = g(x_n)
        except ZeroDivisionError:
            print(f"第 {n} 次迭代: 零除错误，跳过此点")
            return None
        except OverflowError:
            print(f"第 {n} 次迭代: 遇到溢出错误，终止迭代")
            return None
        
        if abs(x_next - x_n) < tau:
            print(f"第 {n} 次迭代: 收敛")
            return x_next
        
        x_n = x_next
    
    print("失败：达到最大迭代次数")
    return None

# 测试函数
def test_g1(x):
    return x**3 - 1

def test_g2(x):
    return (x + 1)**(1/3)

def test_g3(x):
    return 1 / (x**2 - 1)

def test_g4(x):
    return x - (x**3 - x - 1) / (3 * x**3 - 1)


eps = 1e-10  # 很小的容差
test_functions = [test_g1, test_g2, test_g3, test_g4]

for f in test_functions:
    fixed_point = fixed_point_iteration(f, 1 + eps, eps, 20)
    if fixed_point is not None:
        print(f"找到不动点: x = {fixed_point}\n")
    else:
        print("未找到不动点\n")

第 9 次迭代: 遇到溢出错误，终止迭代
未找到不动点

第 14 次迭代: 收敛
找到不动点: x = 1.324717957239529

第 3 次迭代: 零除错误，跳过此点
未找到不动点

第 19 次迭代: 收敛
找到不动点: x = 1.3247179572649885



在证明不动点存在且唯一的条件时，我们看到，若 $p_0\in[a,b]$ 则有 $p_1\in[a,b]$，依此类推，对任意 $k$ 有 $p_k\in[a,b]$. 因为 $$|p_k-p|=|g(p_{k-1})-g(p)|=|g^\prime(\xi_{k-1})(p_{k-1}-p)|\le L|p_{k-1}-p|$$

继续递推 $$|p_{k+1}-p|\le L|p_k-p|$$ 那么 $$|p_{k+1}-p_k|\ge |p_k-p|-|p_{k+1}-p|\ge(1-L)|p_k-p|$$ 由中值定理 $$|p_{k+1}-p_k|\le L|p_k-p_{k-1}|\le\cdots\le L^k|p_1-p_0|$$ 因而 $$|p_k-p|\le\frac{1}{1-L}|p_{k+1}-p_k|\le\frac{L^k}{1-L}|p_1-p_0|$$

当 $L\ge 1$ 不收敛，当 $L$ 接近 1 时收敛很慢，当 $L$ 很小时，收敛很快. 

### Newton 法

#### Newton-Raphson 法

Newton-Raphson 法通过对函数 $f(x)$ 的导数进行分析计算. 如果已知导数，那么就应该选择使用这种快速收敛的方法，尽管不能保证收敛到所需的根.

算法名称来自 Isaac Newton 在 *De analysi per aequationes numero terminorum infinitas*（1669）和 *De metodis fluxionum et serierum infinitarum*（1671）中对该方法的一个特殊情况的描述.

假设 $f \in C^2[a, b]$. 让 $p_0 \in [a, b]$ 为 $p$ 的近似值，使得 $f^\prime(p_0)\ne0$ 且 $|p-p_0|$ 很小. 考虑 $f(x)$ 的一阶 Taylor 多项式围绕 $p_0$ 展开，并在 $x=p$ 处求值.

$$f(p)=f(p_0)+(p-p_0)f^\prime(p_0)+\frac{(p-p_0)^2}{2}f^{\prime\prime}(\xi(p))$$

其中，$\xi(p)$ 介于 $p$ 和 $p_0$ 之间. 由于 $f(p)=0$，上式变为

$$0=f(p_0)+(p-p_0)f^\prime(p_0)+\frac{(p-p_0)^2}{2}f^{\prime\prime}(\xi(p))$$

假设 $|p-p_0|$ 很小，所以 $0\approx f(p_0)+(p-p_0)f^\prime(p_0)$，从而解得

$$p\approx p_0-\frac{f(p_0)}{f^\prime(p_0)}$$

迭代公式如下，

$$p_k = p_{k-1}-\frac{f(p_{k-1})}{f^\prime(p_{k-1})},\quad k\ge 1$$

可以看出，Newton-Raphson 法是不动点迭代 $p_k=g(p_{k-1})$ 的一种特例.

<img src="./assets/newton.png" width="480" style="display: block;margin-left: auto;margin-right: auto;" />

In [5]:
def newton_method(f, df, x0, tau, N):
    x = x0
    iterations = [x0]
    
    for n in range(N):
        fn = f(x)
        dfn = df(x)
        if dfn == 0:
            print("导数为零，迭代失败")
            return None, iterations
        
        x_n = x - fn / dfn
        iterations.append(x_n)
        
        if abs(x_n - x) < tau:
            return x_n, iterations
        
        x = x_n
    
    print("失败：达到最大迭代次数")
    return None, iterations

# 定义测试函数及其导数
def test_f(x):
    return x**2 - 2

def test_df(x):
    return 2*x

x0 = 1
eps = 1e-5
N = 1000

# 使用RootFindingAnimator类
# 包装为适合RootFindingAnimator的lambda函数
animator = RootFindingAnimator(test_f, lambda f, x0, tau, N: newton_method(test_f, test_df, x0, tau, N), x0, eps, N)
animator.animate()

在实际应用中，选择一个初始近似值，然后通过 Newton 法产生连续的近似值. 这些近似值通常要么迅速收敛到根，要么明显不可能收敛. 

- Newton 法收敛快，但不确保收敛.
- 需要对导数求值，通常会消耗大量的计算.

#### Newton 下山法

为了防止 Newton 法发散，可以结合**下山法** (downhill) 保证函数值稳定下降，即附加一个条件 

$$|f(p_{k})|<|f(p_{k-1})|$$

引进**下山因子** $\lambda$，它满足 $0<\lambda \le 1$

$$p_k = \lambda\left(p_{k-1}-\frac{f(p_{k-1})}{f^\prime(p_{k-1})}\right) + (1-\lambda)p_{k-1},\quad k\ge 1$$

在实践中，给定试算初始值，$\lambda$ 通常从 1 开始折半查找. 如果能找到使单调性条件成立的 $\lambda$ 则成功，否则查找失败，需要重新给定初始值.

下山法的核心思想也可以用在其他算法中.

In [6]:
def newton_downhill(f, df, p0, tau, N):
    k = 1
    lam = 1  # 下山因子
    iterations = [p0]  # 记录初始点

    while k <= N:
        p = lam * (p0 - f(p0) / df(p0)) + (1 - lam) * p0
        if abs(f(p)) < abs(f(p0)):
            iterations.append(p)  # 记录当前迭代点
            if abs(p - p0) < tau:
                return p, iterations
            p0 = p
        else:
            lam /= 2  # 折半下山因子
        k += 1
    
    return None, iterations  # 返回None表示未找到根

# 定义测试函数及其导数
def test_f(x):
    return x**2 - 2

def test_df(x):
    return 2*x

p0 = 1
eps = 1e-5
N = 1000

# 使用RootFindingAnimator类
animator = RootFindingAnimator(test_f, lambda f, x0, tau, N: newton_downhill(f, test_df, p0, tau, N), p0, eps, N)
animator.animate()

#### 割线法

用**差商** (difference quotient) 近似代替**导数** (derivative). 根据定义

$$f^\prime(p_{k-1})=\lim_{x\to p_{k-1}}\frac{f(x)-f(p_{k-1})}{x-p_{k-1}}$$

如果 $p_{k-2}$ 与 $p_{k-1}$ 充分接近，则有

$$f^\prime(p_{k-1})\approx\frac{f(p_{k-2})-f(p_{k-1})}{p_{k-2}-p_{k-1}}=\frac{f(p_{k-1})-f(p_{k-2})}{p_{k-1}-p_{k-2}}$$

代入 Newton 法的式子，得

$$p_k=p_{k-1}-\frac{f(p_{k-1})(p_{k-1}-p_{k-2})}{f(p_{k-1})-f(p_{k-2})}$$

这就是**割线法** (secant method) 的迭代公式.

<img src="./assets/secant.png" width="480" style="display: block;margin-left: auto;margin-right: auto;" />

In [7]:
def secant_method(f, x_0, x_1, tau, N):
    iterations = [x_0, x_1]  # 记录初始点

    for n in range(N):
        f_x_0 = f(x_0)
        f_x_1 = f(x_1)
        
        if (f_x_1 - f_x_0) == 0:
            print("失败：割线斜率为零")
            return None, iterations
        
        x_next = x_1 - f_x_1 * (x_1 - x_0) / (f_x_1 - f_x_0)
        iterations.append(x_next)  # 记录当前迭代点

        if abs(x_next - x_1) < tau:
            return x_next, iterations
        
        x_0, x_1 = x_1, x_next
    
    print("失败：达到最大迭代次数")
    return None, iterations

# 定义测试函数
def test_f(x):
    return x**2 - 2

x_0 = 0
x_1 = 2
eps = 1e-5
N = 1000

# 使用RootFindingAnimator类
animator = RootFindingAnimator(test_f, lambda f, x0, tau, N: secant_method(f, x_0, x_1, tau, N), x_0, eps, N)
animator.animate()

#### 松弛 Newton 法

通过引入一个松弛因子 $\omega$ ($0<\omega\le 1$) 来调整每一步的更新幅度，可以在某些情况下提高收敛的稳定性或解决发散问题. 迭代公式如下，

$$p_k = p_{k-1}-\omega\frac{f(p_{k-1})}{f^\prime(p_{k-1})},\quad k\ge 1$$

- 当 $\omega < 1$ 时，步长缩小
- 当 $\omega = 1$ 时，带松弛因子的牛顿法退化为标准 Newton 法

标准 Newton 法通常在初始猜测接近实际根时收敛非常快，但如果初始猜测离根较远或函数曲率较大，可能会导致发散或跳根. 

松弛 Newton 法通过缩小步长，松弛因子可以避免过大步长带来的发散问题. 在某些函数中（例如高度非线性或多根的函数），$\omega < 1$ 可以使算法更加稳定.

In [8]:
def newton_method_relaxed(f, df, x_0, tau, omega, N):
    x_n = x_0
    iterations = [x_0]  # 记录初始点

    for n in range(N):
        f_n = f(x_n)
        df_n = df(x_n)
        if df_n == 0:
            print("失败：导数为零")
            return None, iterations
        
        # 应用松弛因子调整迭代步长
        x_next = x_n - omega * f_n / df_n
        iterations.append(x_next)  # 记录当前迭代点

        if abs(x_next - x_n) < tau:
            return x_next, iterations
        
        x_n = x_next
    
    print("失败：达到最大迭代次数")
    return None, iterations

# 定义一个测试函数及其导数
def test_f(x):
    return x**2 - 2

def test_df(x):
    return 2*x

omega = 0.5  # 松弛因子
x_0 = 1
eps = 1e-5
N = 1000

# 使用RootFindingAnimator类
animator = RootFindingAnimator(test_f, lambda f, x0, tau, N: newton_method_relaxed(f, test_df, x_0, tau, omega, N), x_0, eps, N)
animator.animate()

当算法接近根时，误差 $|f(p_k)|$ 会变得很小，我们可以根据这个误差来动态调整松弛因子 $\omega$. 例如，当误差小于某个阈值时，可以减小它.

In [9]:
def newton_method_relaxed_dynamic(f, df, x_0, tau, omega_initial, N):
    x_n = x_0
    omega = omega_initial  # 初始松弛因子
    iterations = [x_0]  # 记录初始点

    for n in range(N):
        f_n = f(x_n)
        df_n = df(x_n)
        if df_n == 0:
            print("失败：导数为零")
            return None, iterations
        
        # 动态调整松弛因子
        if abs(f_n) < 1e-2:  # 误差较小时减小松弛因子
            omega = max(0.1, omega / 2)
        
        # 应用松弛因子调整迭代步长
        x_next = x_n - omega * f_n / df_n
        iterations.append(x_next)  # 记录当前迭代点

        if abs(x_next - x_n) < tau:
            return x_next, iterations
        
        x_n = x_next
    
    print("失败：达到最大迭代次数")
    return None, iterations

# 定义一个测试函数及其导数
def test_f(x):
    return x**2 - 2

def test_df(x):
    return 2*x

omega_initial = 0.8  # 初始松弛因子
x_0 = 1
eps = 1e-5
N = 1000

# 使用RootFindingAnimator类
animator = RootFindingAnimator(test_f, lambda f, x0, tau, N: newton_method_relaxed_dynamic(f, test_df, x_0, tau, omega_initial, N), x_0, eps, N)
animator.animate()

### 收敛

假设 $\{p_k\}_{k=0}^\infty$ 是一个收敛于 $p$ 的序列，对所有 $k$ 而言，$p_k\ne p$. 如果存在正常数 $\lambda$ 和 $\alpha$，且 $$\lim_{k\to\infty} \frac{|p_{k+1} -p|}{|p_k-p|^\alpha} =\lambda$$则 $\{p_k\}_{k=0}^\infty$ 以 $\alpha$ 阶收敛于 $p$，**渐近误差常数** (asymptotic error constant) 为 $\lambda$.

如果序列 $\{p_k\}_{k=0}^\infty$ 以 $\alpha$ 阶收敛于解 $p=g(p)$，则称形如 $p_k=g(p_{k-1})$ 的迭代技术为 $\alpha$ 阶.

> **不动点定理** (Fixed-Point Theorem)：设 $g\in C[a,b]$，且对于所有 $x\in[a,b]$ 满足 $g(x)\in[a,b]$. 此外，假设 $g^\prime$ 在 $(a,b)$ 上存在，并且存在一个常数 $0<L<1$，其中 $$|g^\prime(x)|\le L,\quad x\in(a,b)$$ 那么，对于 $[a,b]$ 中的任意数 $p_0$，由式 $$p_k=g(p_{k-1}),\quad k\ge 1$$ 定义的序列收敛于 $[a,b]$ 中的唯一不动点 $p$.

> **不动点迭代的收敛阶数**：设 $g\in C[a,b]$，且对于所有 $x\in[a,b]$ 满足 $g(x)\in[a,b]$. 此外，假设 $g^\prime$ 在$(a,b)$ 上连续，并且存在一个常数 $0<L<1$，使得 $$|g^\prime(x)|\le L,\quad x\in(a,b)$$ 如果 $g^\prime(p)\ne 0$，那么对于 $[a,b]$ 中的任意数 $p_0\ne p$，序列 $$p_k = g(p_{k-1}),\quad k\ge 1$$仅线性收敛到 $[a,b]$ 中的唯一不动点 $p$.

不动点迭代的线性收敛定理暗示，只有当 $g^\prime(p)=0$ 时，形如 $g(p)=p$ 的不动点迭代收敛阶数才是超线性的. 即，若一个不动点迭代技术是二阶收敛的，必须同时满足 $g(p)=p$ 和 $g^\prime(p)=0$.

考虑序列$$p_k=g(p_{k-1}),\quad k\ge 1$$其中 $g$ 满足$$g(x)=x-\phi(x)f(x)$$
式中 $\phi$ 是一个待确定的可微函数. 又知 $$g^\prime(x)=1-\phi^\prime(x)f(x)-f^\prime(x)\phi(x)$$ 和 $f(p)=0$，我们有$$g^\prime(p) = 1-\phi^\prime(p)f(p)-f^\prime(p)\phi(p) = 1-\phi^\prime(p)\cdot0-f^\prime(p)\phi(p)=1-f^\prime(p)\phi(p)$$

对照目标，当且仅当 $\phi(p)=1/f^\prime(p)$ 时，$g^\prime(p)=0$.

因此，取 $\phi(p)=1/f^\prime(p)$，代入迭代式子$$p_k=g(p_{k-1})=p_{k-1}-\frac{f(p_{k-1})}{f^\prime(p_{k-1})}$$
此为 Newton 法.

Newton 法的收敛阶数是 2. 证明如下

> 设 $p$ 是根，$p_k$ 是根的第 $k$ 个近似值. 将绝对误差界定义为$$|\varepsilon_k|=|p-p_k|$$ 对于足够大的 $k$，有近似关 $$|\varepsilon_{k+1}|=\lambda|\varepsilon_k|^\alpha$$ 将 Newton 法迭代式写成 $$p_{k+1}=p_k-\frac{f(p_k)}{f^\prime(p_k)}$$其中根 $p$ 满足 $f(p)=0$. 利用迭代公式减去两边，我们有$$p - p_{k+1}=p - p_k+\frac{f(p_k)}{f^\prime(p_k)}$$ 或写作
$$\varepsilon_{k+1}=\varepsilon_k+\frac{f(p_k)}{f^\prime(p_k)}$$ 用 Taylor 级数来展开关于根 $p$ 的函数 $f(p_k)$ 和 $f^\prime(p_k)$，使用 $f(p)=0$，有 $$\begin{align*}
f(p_k)&=f(p)+(p_k-p)f^\prime(p)+\frac{1}{2}(p_k-p)^2f^{\prime\prime}(p)+\dots\\
&=-\varepsilon_kf^\prime(p)+\frac{1}{2}\varepsilon_k^2f^{\prime\prime}(p)+\dots;\\
f^\prime(p_k)&=f^\prime(p)+(p_k-p)f^{\prime\prime}(p)+\frac{1}{2}(p_k-p)^2f^{\prime\prime\prime}(p)+\dots\\
&=f^\prime(p)-\varepsilon_kf^{\prime\prime}(p)+\frac{1}{2}\varepsilon_k^2f^{\prime\prime\prime}(p)+\dots
\end{align*}$$ 利用以下标准 Taylor 级数：$$\frac{1}{1-\varepsilon}=1+\varepsilon+\varepsilon^2+\dots$$它对于 $|\varepsilon|<1$ 收敛. 将这些 Taylor 展开式代入上面的误差式子，得到 $$\begin{align*}
\varepsilon_{k+1}&=\varepsilon_k+\frac{f(p_k)}{f^\prime(p_k)}\\
&=\varepsilon_k+\frac{-\varepsilon_kf^\prime(p)+\frac{1}{2}\varepsilon_k^2f^{\prime\prime}(p)+\dots}{f^\prime(p)-\varepsilon_kf^{\prime\prime}(p)+\frac{1}{2}\varepsilon_k^2f^{\prime\prime\prime}(p)+\dots}\\
&=\varepsilon_k+\frac{-\varepsilon_k+\frac{1}{2}\varepsilon_k^2\frac{f^{\prime\prime}(p)}{f^{\prime}(p)}+\dots}{1-\varepsilon_k\frac{f^{\prime\prime}(p)}{f^{\prime}(p)}+\dots}\\&=\varepsilon_k+\left(-\varepsilon_k+\frac{1}{2}\varepsilon_k^2\frac{f^{\prime\prime}(p)}{f^{\prime}(p)}+\dots\right)\left(1+\varepsilon_k\frac{f^{\prime\prime}(p)}{f^{\prime}(p)}+\dots\right)\\
&=\varepsilon_k+\left(-\varepsilon_k-\varepsilon_k^2\frac{f^{\prime\prime}(p)}{f^{\prime}(p)}+\frac{1}{2}\varepsilon_k^2\frac{f^{\prime\prime}(p)}{f^{\prime}(p)}+\dots\right)\\
&=-\frac{1}{2}\frac{f^{\prime\prime}(p)}{f^{\prime}(p)}\varepsilon_k^2+\dots\end{align*}$$ 因此，证明了随着 $k\to\infty$，$$|\varepsilon_{k+1}|=\lambda|\varepsilon_k|^2$$ 只要 $f^\prime(r)\ne 0$，$k$ 满足$$k=\frac{1}{2}\left|\frac{f^{\prime\prime}(p)}{f^{\prime}(p)}\right|$$ 牛顿方法在单根上是二阶的，即**平方收敛** (quadratically convergent).



